In [1]:
import numpy as np
import pennylane as qml
from pennylane import X, Y, Z, I
from kak_tools import identify_algebra, split_pauli_algebra, map_simple_to_irrep, map_irrep_to_matrices

In [2]:
def close_and_split(gens, verbose=True):
    gens = [list(op.pauli_rep)[0] for op in gens] # Map generators from Operator to PauliWord
    dla = qml.lie_closure(gens, pauli=True)
    dla = [list(op)[0] for op in dla] # Map dla from PauliSentence to PauliWord
    np.random.shuffle(dla)
    sub_dlas = split_pauli_algebra(dla, verbose=verbose)
    return dla, sub_dlas

In [3]:
# Test with so(2n)
n = 8
m = 7
gens = [X(w) @ X(w+1) for w in range(n-1)] + [Y(w) @ Y(w+1) for w in range(n-1)] + [Z(w) for w in range(n)]
gens += [X(w) @ Y(w+1) for w in range(n, m+n-1)]
dla, sub_dlas = close_and_split(gens)
# print(sub_dlas[0])
# print(sub_dlas[1])
identifiers = identify_algebra(sub_dlas, verbose=True)
print(identifiers)

Found 2 components with dimensions [120, 21].
Dimension of component: 120.
Component 0 can be one of the following:
so(16)
8 copies of so(6)
Dimension of component: 21.
Component 1 can be one of the following:
so(7)
sp(3)
[[(1, 'so', 16), (8, 'so', 6)], [(1, 'so', 7), (1, 'sp', 3)]]


In [4]:
# # Test with random generators
# def make_random_semisimple_algebra(num_wires=5, num_gens=7, seed=None):
#     rng = np.random.default_rng(seed=seed)
#     failure = True
#     while failure:
#         words = rng.choice([I, X, Y, Z], replace=True, size=(num_gens, num_wires))
#         gens = [qml.prod(*(P(w) for w, P in enumerate(word))) for word in words]
#         dla, sub_dlas = close_and_split(gens, verbose=False)
#         dims = [len(sub) for sub in sub_dlas]
#         if 1 not in dims:
#             failure = False
#     return dla
    
# num_wires = 8
# num_gens = 10
# seed = 2
# gens = make_random_semisimple_algebra(num_wires, num_gens, seed=seed)
# # print(f"Max DLA dimension: {4**num_wires-1}")
# dla, sub_dlas = close_and_split(gens)
# # print(sub_dlas)
# # identifiers = identify_algebra(sub_dlas, verbose=True)

## Map to irrep and associated matrices

In [13]:
import numpy as np
from pennylane import X, Y, Z, I, lie_closure
from kak_tools import map_simple_to_irrep, map_irrep_to_matrices


n = 11
n_so = 2 * n
sub_hor_size = 3 * n

so_dim = (n_so**2-n_so) // 2
print(f"{n=}; so(2n) dim: {so_dim}")

gens = [X(w) @ X(w+1) for w in range(n-1)] + [Y(w) @ Y(w+1) for w in range(n-1)] + [Z(w) for w in range(n)]
gens = [next(iter(op.pauli_rep)) for op in gens]

dla = lie_closure(gens, pauli=True)
dla = [next(iter(op.pauli_rep)) for op in dla]
assert len(dla) == so_dim

def theta(op):
    """Even Odd"""
    return -(-1) ** sum(p=="Y" for p in op.values())


if sub_hor_size > (n_so//2) ** 2:
    raise ValueError("Not enough room in the horizontal space")

hor_gens = np.random.choice([op for op in dla if theta(op) == -1], replace=False, size=sub_hor_size)
H_coeffs = np.random.normal(0, 1., sub_hor_size)

n=11; so(2n) dim: 231


In [14]:
mapping = map_simple_to_irrep(dla, hor_gens, n=n_so, invol_type="BDI")
matrix_map = map_irrep_to_matrices(mapping, n_so, invol_type="BDI")
matrices_in_order = np.stack([matrix_map[op] for op in dla])

In [15]:
H = H_coeffs @ np.stack([matrix_map[op] for op in hor_gens])

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 22 is different from 33)